In [1]:
%cd ..

d:\project\deploy-chest-X-ray


In [2]:
from PIL import Image

In [3]:
Image.open

<function PIL.Image.open(fp: 'StrOrBytesPath | IO[bytes]', mode: "Literal['r']" = 'r', formats: 'list[str] | tuple[str, ...] | None' = None) -> 'ImageFile.ImageFile'>

In [4]:
from pathlib import Path 
import os 
import pandas as pd
from glob import glob

In [5]:
folder_path = r"data\train\NORMAL"
imgs_paths = glob(folder_path + r"\*.jpeg")
len(imgs_paths)

1341

In [6]:

def build_data_map(folder_path):
    image_path = glob(folder_path + r"\*.jpeg")

    clas = Path(folder_path).name
    split = Path(folder_path).parent.name

    return pd.DataFrame({
        "image": image_path,
        "class": clas,
        "split": split
        })

In [7]:


test_normal = build_data_map(r"data\test\NORMAL")
test_pnem = build_data_map(r"data\test\PNEUMONIA")

train_normal = build_data_map(r"data\train\NORMAL")  #1341 images number
train_pnem = build_data_map(r"data\train\PNEUMONIA") #3875 images number

val_normal = build_data_map(r"data\val\NORMAL") #8 images number
val_pnem = build_data_map(r"data\val\PNEUMONIA") #8 images number

In [8]:
print(len(train_normal))
print("%"*22)
print(len(train_pnem))
print("%"*22)
print(len(val_normal))
print("%"*22)
print(len(val_pnem))

1341
%%%%%%%%%%%%%%%%%%%%%%
3875
%%%%%%%%%%%%%%%%%%%%%%
8
%%%%%%%%%%%%%%%%%%%%%%
8


In [9]:
train_pnem_balance = train_pnem.sample(n=1341,replace=True)

In [10]:
data = pd.concat([
    test_normal,
    test_pnem,
    train_normal,
    train_pnem_balance,
    val_normal,
    val_pnem
])

In [11]:
data

,image,class,split
0,data\test\NORMAL\IM-0001-0001.jpeg,NORMAL,test
1,data\test\NORMAL\IM-0003-0001.jpeg,NORMAL,test
2,data\test\NORMAL\IM-0005-0001.jpeg,NORMAL,test
3,data\test\NORMAL\IM-0006-0001.jpeg,NORMAL,test
4,data\test\NORMAL\IM-0007-0001.jpeg,NORMAL,test
...,...,...,...
3,data\val\PNEUMONIA\person1949_bacteria_4880.jpeg,PNEUMONIA,val
4,data\val\PNEUMONIA\person1950_bacteria_4881.jpeg,PNEUMONIA,val
5,data\val\PNEUMONIA\person1951_bacteria_4882.jpeg,PNEUMONIA,val
6,data\val\PNEUMONIA\person1952_bacteria_4883.jpeg,PNEUMONIA,val


In [13]:
from tqdm import tqdm
resized_img_path=[]
for i, row in tqdm(data.iterrows(),total=data.shape[0]):
    image_path = row["image"]
    image = Image.open(image_path)
    resized_img = image.resize((128,128))

    img_path = Path(image_path)
    img_name = img_path.name
    img_clas = img_path.parent.name
    img_split = img_path.parent.parent.name

    final_img_name = img_split + "_" + img_clas + "_" + img_name
    final_img_path = r"data/resize/" + final_img_name

    resized_img.save(final_img_path)
    resized_img_path.append(final_img_path)

100%|██████████| 3322/3322 [02:44<00:00, 20.21it/s]


In [14]:
data['resized']=resized_img_path

In [ ]:
data.to_csv("./data_map.csv",index=False)